In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import cv2

## 

# Select which video to analize

Replace INPUT_VIDEO_PATH var below with the name of video you want to analyze

In [3]:
#INPUT_VIDEO_PATH = "input_files/Traffic_Laramie_1.mp4"
INPUT_VIDEO_PATH = "input_files/Traffic_Laramie_2.mp4"

# Set hyperparameters

In [4]:

CONSECUTIVE_FRAMES = 2 # how manny frames will be compared to detect movement, less = more detailed detection, more computation needed
MINIMAL_SIZE = 1600 # minimal object size, big enough to detect cars but avoid pedestrians

# Define function that generates background model

The below function extracts the background from a video and returns a frame with all moving objects removed. To do this it obtains the median over 50 random frames selected from the video.

In [5]:
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # we will randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    return median_frame

# Read video and instantiate video writer

Reads input video and extracts width, height and fps.
then uses those parameters to instantiate a video writer that will be used to write the updated frames to disk.

In [6]:
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
# get the video frame height and width
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

save_name = f"output_files_task2/{INPUT_VIDEO_PATH.split('/')[-1]}"
# define codec and create VideoWriter object
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*'mp4v'), fps, 
    (frame_width, frame_height)
)

# Get background frame

Applies background substraction technique to obtain a frame with moving objects removed

In [7]:
# get the background model
background = get_background(INPUT_VIDEO_PATH)
# convert the background model to grayscale format
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)


# Instantiate important utility functions

**reset_counters()**: resets counters used for tracking car count  

**find_closest_cord(cord, cords_set)**: finds closest point (and distance) betweenn given coordinates and cords_set. to do this compares euclidean distance between given coordinates and all points in cords_set.  

**update_cord(current_cords, new_cors, cords_set)**: replaces old cords in cords_set with updated cords.  

**draw_contours(contours, frame)**: draws bounding box on frame if object size is above MINIMAL_SIZE, also updates the car counter by tracking object's coordinate changes in cords_set


In [8]:
import numpy as np
import time
car_count = 0
MAX_OBJECT_DISTANCE = 60
cords_set = set()
cords_to_delete_map = {}

def reset_counters():
    global car_count
    global cords_set
    car_count = 0
    cords_set = set()

def find_closest_cord(cord, cords_set):
    closest_dist = 100000000
    closest_cord = None
    for c in cords_set:
        point1 = np.array(cord)
        point2 = np.array(c)
        dist = np.linalg.norm(point1 - point2)
        if(dist < closest_dist):
            closest_dist = dist
            closest_cord = c 
    return closest_cord, closest_dist

def update_cord(current_cords, new_cors, cords_set):
    cords_set.remove(current_cords)
    cords_set.add(new_cors)

def draw_contours(contours, frame):
    global car_count
    global cords_set
    global cords_to_delete_map
    cords_in_frame = set()
    for contour in contours:
        # continue through the loop if contour area is less than MINIMAL_SIZE...
        # ... helps in removing noise detection
        if cv2.contourArea(contour) < MINIMAL_SIZE:
            continue
        # get the xmin, ymin, width, and height coordinates from the contours
        (x, y, w, h) = cv2.boundingRect(contour)
        X_THRESHOLD = 150
        if(y>280 and x<500 and ((x<X_THRESHOLD and y < 370) or (x>=X_THRESHOLD and y<400))):
            ### object tracking starts here
            cords_in_frame.add((x, y))
            closest_cord, distance = find_closest_cord((x, y), cords_set)
            if(distance <= MAX_OBJECT_DISTANCE):
                update_cord(closest_cord, (x,y), cords_set)
            else:
                car_count += 1
                cords_set.add((x, y))
            ### object tracking ends here
            # draw the bounding boxes
            cv2.rectangle(frame, (x, y+2), (x+w, y+h), (0, 255, 0), 2)
    ### remove old cords from cords_set
    for cord in list(cords_set):
        closest_cord, distance = find_closest_cord(cord, cords_in_frame)
        if(distance > MAX_OBJECT_DISTANCE):
            key = f"{cord[0]}_{cord[1]}"
            if key not in cords_to_delete_map:
                 cords_to_delete_map[key] = 0
            cords_to_delete_map[key] +=1
            if(cords_to_delete_map[key] > 3):
                del cords_to_delete_map[key] 
                print("deleting cord from cords set(is out of frame): ", key)
                cords_set.remove(cord)
    ##### END old cord removal
    cv2.putText(frame, f"Car count: {str(car_count)} ", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
           1, (0, 255, 0), 2, cv2.LINE_AA) 

        

# Create mask to remove all but main street

Below I will generate a frame to use it as a mask, all pixels will be 0 except for pixels that cover the main street area which I will set to 255, to apply it we will later perform an and operation between the mask and each video frame.

In [9]:
### mask only street we care about

# a mask is the same size as our image, but has only two pixel
# values, 0 and 255 -- pixels with a value of 0 (background) are
# ignored in the original image while mask pixels with a value of
# 255 (foreground) are allowed to be kept
mask = np.zeros((frame_height, frame_width), dtype="uint8")
mask2 = np.zeros((frame_height, frame_width), dtype="uint8")
mask3 = np.zeros((frame_height, frame_width), dtype="uint8")
cv2.rectangle(mask, (0, 200), (80, 400), 255, -1)
cv2.rectangle(mask2, (80, 200), (250, 420), 255, -1)
cv2.rectangle(mask3, (250, 200), (500, 450), 255, -1)
mask4 = cv2.bitwise_or(cv2.bitwise_or(mask, mask2), mask3)

# Main loop: iterate over frames and perform object detection and object tracking

Loop over input video frames and do the following:  
1- convert frames to grayscale for easier detection  
2- find difference between current frame and background frame, ie: remove the background.  
3- apply a threshold to convert all pixels to either 1 or 0 depending on difference with background.  
4- Apply mask to only pay attention to objects on main street  
5- Sum consecutive frames(with background removed): find contours around objects, represented as group of pixels clustered together with values above 0, only reason to do this is to reduce computation. finding the contour on a sequence of CONSECUTIVE_FRAMES frames is CONSECUTIVE_FRAMES times faster than doing it for individual frames.  
6- Use OpenCV's findContours function to find contours for consecutive pixels that have non 0 values. We only find contours for the last pixel of each group of CONSECUTIVE_FRAMES, since need the previous frames of the group to calculate it(Later I still use the same contour for every pixel in the group)  
7- append each frame with detected contours(if available) to a list of frames to later be written to disk.

In [10]:
reset_counters()
frame_diff_list = []
frames_to_write = []
frame_count = 0
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame_count += 1
        orig_frame = frame.copy()
        # IMPORTANT STEP: convert the frame to grayscale first
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if frame_count % CONSECUTIVE_FRAMES == 0 or frame_count == 1:
            frame_diff_list = []
        # find the difference between current frame and base frame
        frame_diff = cv2.absdiff(gray, background)
        # thresholding to convert the frame to binary
        ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        
        # apply our mask
        thres = cv2.bitwise_and(thres, thres, mask=mask4)
            
        
        # dilate the frame a bit to get some more white area...
        # ... makes the detection of contours a bit easier
        dilate_frame = cv2.dilate(thres, None, iterations=2)

        # append the final result into the `frame_diff_list`
        frame_diff_list.append(dilate_frame)
        # if we have reached `CONSECUTIVE_FRAMES` number of frames
        if len(frame_diff_list) == CONSECUTIVE_FRAMES:
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)
            
            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            frames_to_write.append((orig_frame, contours))
        else:
            # write intermediate frames to avoid frame skipping in output video? 
            frames_to_write.append((orig_frame, []))
    else:
        break

# write frames with the contours to disk

Loop over frames, contours list processed in previous step and for each frame:
1) If don't have contours for this image, find the next contour in the list
2) draw all contours as rectangles in the frame
2) Write frame to disk

In [11]:
reset_counters()
i=0
for frame, contours in frames_to_write:
    j=1
    while(not contours and (i+j) <len(frames_to_write)):
        contours = frames_to_write[i+j][1]
        j+=1
    i+=1
    draw_contours(contours, frame)
    out.write(frame)


deleting cord from cords set(is out of frame):  0_332
deleting cord from cords set(is out of frame):  0_350
deleting cord from cords set(is out of frame):  0_341
deleting cord from cords set(is out of frame):  0_342


# Calculate cars per minute and print total car count

In [12]:
total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration_sec = total_frame_count/fps
number_of_minutes = duration_sec/60
cars_per_minute = car_count / number_of_minutes
print("cars per minute: ",cars_per_minute)
print("total car count: ", car_count)

cars per minute:  2.271006813020439
total car count:  4


In [13]:
number_of_minutes, fps, total_frame_count

(1.7613333333333334, 25, 2642)

# Close video reader and writer

In [14]:
cap.release()
out.release()